In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from src.managers.cross_validation import TimeSeriesCrossValidator

# Load the dataset
df = pd.read_csv('src/managers/data/processed_data.csv', index_col="Date", parse_dates=True)
df.head()

,CPIAUCSL,VIXCLS,TEDRATE,BAA10Y,DSPIC96,A229RX0,CMRMTSPL,INDPRO,IPFPNSS,IPFINAL,...,DGS5_6_mo_return_direction,DGS7_1_mo_return_direction,DGS7_3_mo_return_direction,DGS7_6_mo_return_direction,DGS1_1_mo_return_direction,DGS1_3_mo_return_direction,DGS1_6_mo_return_direction,DGS20_1_mo_return_direction,DGS20_3_mo_return_direction,DGS20_6_mo_return_direction
Date,,,,,,,,,,,,,,,,,,,,,
1998-01-31,162.0,21.47,0.57,1.68,9022.9,32843.0,953719.0,84.1564,93.2884,91.3778,...,0,0,0,0,0,0,0,0,0,0
1998-02-28,162.0,18.55,0.50,1.67,9081.2,33029.0,967109.0,84.3066,93.5021,91.4979,...,0,1,0,0,1,0,0,1,0,0
1998-03-31,162.0,24.22,0.69,1.67,9134.4,33195.0,971898.0,84.3420,93.6295,91.6173,...,0,1,0,0,0,0,0,1,0,0
1998-04-30,162.2,21.18,0.85,1.66,9164.1,33271.0,973216.0,84.6339,94.0290,92.0431,...,0,0,1,0,0,1,0,1,1,0
1998-05-31,162.6,21.32,0.80,1.63,9202.0,33377.0,976540.0,85.1867,94.5829,92.4608,...,0,0,0,0,1,1,0,0,0,0


## Applying XGBoost to Fixed Income Signals Prediction

#### Overview

**Task**: Predict the direction in 1, 3, and 6 month intervals for key asset classes across the Fixed Income space. These asset classes are:
- US Treasuries (1, 2, 5, 7, 10, 20, and 30 year durations)
- US High Yield Bonds
- US Investment Grade Bonds

**Method**: Utilize rolling window 